In [2]:
import sys
print(sys.executable)
%load_ext autoreload
%autoreload 2
import os
import yaml
import getpass
import yaml
import logging
import glob
import requests
import pandas as pd
import math
import matplotlib.pyplot as plt
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.core.display import display, HTML
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import re
import copy
import logging
logger = logging.getLogger()
logging.basicConfig(format='%(message)s', level=logging.DEBUG)
# from common import helper_functions
#from common import pupil
%matplotlib notebook
display(HTML('<style>.container { width:95% !important; }</style>'))
pd.options.display.max_columns= 999
init_notebook_mode(connected=True)

Loaded backend nbAgg version unknown.


C:\Users\varad\anaconda3\python.exe
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import plotly
from sklearn import preprocessing
import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import cross_val_predict

import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.express as px

from fbprophet import Prophet

%matplotlib inline
plt.style.use('seaborn-white')

Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [4]:
data='train.csv'
df=pd.read_csv(data)
df['Date']=pd.to_datetime(df['Date'])
df.head(5)

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [5]:
def get_province(state,country):
    try:
        if np.isnan(state):
            return country
        else:
            return state
    except:
        return country
df['Province_State']=df.apply(lambda x:get_province(x['Province_State'],x['Country_Region']),axis=1)

In [6]:
start_data=[]
plot_data=[]
fig=tools.make_subplots()
n_unique_provice=df['Province_State'].nunique()
average_length=[]
cur=0
for d,group in df.groupby(['Country_Region','Province_State']):
    country=d[0]
    province=d[1]
#     print(province,country,cur,n_unique_provice)
    temp={}
    temp_df=group.copy().reset_index(drop=True)
    temp['state']=province
    temp['country']=country
    for i,r in temp_df.iterrows():
        if i:
            if r['ConfirmedCases'] > 1:
                temp['start']=r['Date']
                temp['start_index']=i
                break
    start_data.append(temp)
    if('start_index' in temp):
        average_length.append(len(group)-temp['start_index'])
        fig.append_trace(go.Scatter(y=temp_df.loc[temp['start_index']:temp['start_index']+63,'ConfirmedCases'].tolist(),
                                    name=province
                                    ),1,1)
    cur+=1
fig['layout']={
    'xaxis':{'title':'Date',},
    'yaxis':{'title':'Confirmed Cases','type':'log'},
    'title':"Comparision of regions since confirmed cases increased beyond 1"
}
# plot(fig,filename='start-date-fixed.html')
fig.show()

C:\Users\varad\anaconda3\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [ ]:
np.mean(average_length)

# VIS 2

In [9]:
# fig=tools.make_subplots(1,3,subplot_titles=['Confirmed Cases','Fatalities','Fatalities w.r.t. Confirmed Cases'])
# data=[]
# for country,group in df.groupby(['Country_Region']):
#     temp={}
#     temp['Country']=country
#     temp['Confirmed']=group['ConfirmedCases'].tolist()[-1]
#     temp['Fatalities']=group['Fatalities'].tolist()[-1]
#     temp['FatalitiesPercent']=100*temp['Fatalities']/temp['Confirmed']
#     data.append(temp)
# colors=px.colors.qualitative.Plotly
# country_df=pd.DataFrame(data)
# country_df=country_df.sort_values('Confirmed',ascending=False)

# show_top_n=10

# fig.append_trace(go.Bar(x=country_df["Confirmed"].head(show_top_n),
#                         y=country_df["Country"].head(show_top_n), 
#                         name='Confirmed',
#                         orientation='h',
#                         marker_color=colors[0]),1,1)
# fig.append_trace(go.Bar(x=country_df["Fatalities"].head(show_top_n), 
#                         y=country_df["Country"].head(show_top_n), 
#                         name='Fatalities', 
#                         orientation='h',
#                         marker_color=colors[1]),1,2)

# fig.append_trace(go.Bar(x=country_df["Confirmed"].head(show_top_n), 
#                         y=country_df["Country"].head(show_top_n), 
#                         name='Confirmed',
#                         orientation='h',
#                         marker_color=colors[0]),1,3)
# fig.append_trace(go.Bar(x=country_df["Fatalities"].head(show_top_n), 
#                         y=country_df["Country"].head(show_top_n), 
#                         name='Fatalities', 
#                         orientation='h',
#                         marker_color=colors[1]),1,3)

# annotations = []
# index=0
# for c,f in zip(country_df["Confirmed"].head(show_top_n),country_df["Fatalities"].head(show_top_n)):
#     percent=100*f/c
#     print(c,f,percent,index)
#     annotations.append(
#         dict(xref='x3', yref='y1',
#              y=index, x=c+f+25000,
#              text=f'{percent:2.2f}%',
#              showarrow=False)
#                       )
#     index+=1
# #     if index==3:
# #         break


# fig.update_layout(title='',
#                   annotations=annotations,
#                   xaxis={'title':'Confirmed Cases'},
#                   xaxis2={'title':'Confirmed Cases'},
#                   xaxis3={'title':'Confirmed Cases','range':[0,300000]},
#                   barmode='stack',
#                   yaxis = {'autorange':"reversed",'title':'Country'},
#                   yaxis2 = {'autorange':"reversed"},
#                   yaxis3 = {'autorange':"reversed"},
#                  )

# fig.show()
# # plot(fig,filename='country_grouped.html')
# # country_df.head(10)

In [8]:
fig=tools.make_subplots(1,3,subplot_titles=['Confirmed Cases','Fatalities','Fatalities w.r.t. Confirmed Cases'])
data=[]
for country,group in df.groupby(['Country_Region']):
    temp={}
    temp['Country']=country
    temp['Confirmed']=group['ConfirmedCases'].tolist()[-1]
    temp['Fatalities']=group['Fatalities'].tolist()[-1]
    temp['FatalitiesPercent']=100*temp['Fatalities']/temp['Confirmed']
    data.append(temp)
colors=px.colors.qualitative.Plotly
country_df=pd.DataFrame(data)
country_df=country_df.sort_values('Confirmed',ascending=False)

show_top_n=10

fig.append_trace(go.Bar(x=country_df["Confirmed"].head(show_top_n),
                        y=country_df["Country"].head(show_top_n), 
                        name='Confirmed',
                        orientation='h',
                        marker_color=colors[0]),1,1)
fig.append_trace(go.Bar(x=country_df["Fatalities"].head(show_top_n), 
                        y=country_df["Country"].head(show_top_n), 
                        name='Fatalities', 
                        orientation='h',
                        marker_color=colors[1]),1,2)

fig.append_trace(go.Bar(x=country_df["Confirmed"].head(show_top_n), 
                        y=country_df["Country"].head(show_top_n), 
                        name='Confirmed',
                        orientation='h',
                        marker_color=colors[0]),1,3)
fig.append_trace(go.Bar(x=country_df["Fatalities"].head(show_top_n), 
                        y=country_df["Country"].head(show_top_n), 
                        name='Fatalities', 
                        orientation='h',
                        marker_color=colors[1]),1,3)

annotations = []
index=0
for c,f in zip(country_df["Confirmed"].head(show_top_n),country_df["Fatalities"].head(show_top_n)):
    percent=100*f/c
    print(c,f,percent,index)
    annotations.append(
        dict(xref='x3',
             yref='y1',
             y=index,
             x=c+f+25000,
             text=f'{percent:2.2f}%',
             showarrow=False
            )
    )
    index+=1
#     if index==3:
#         break


fig.update_layout(title='',
                  annotations=annotations[:3]+annotations,
                  xaxis={'title':'Confirmed Cases'},
                  xaxis2={'title':'Confirmed Cases'},
                  xaxis3={'title':'Confirmed Cases','range':[0,300000]},
                  barmode='stack',
                  yaxis = {'autorange':"reversed",'title':'Country'},
                  yaxis2 = {'autorange':"reversed","visible":False},
                  yaxis3 = {'autorange':"reversed","visible":False},
                 )

fig.show()
# plot(fig,filename='country_grouped.html')
# country_df.head(10)

242271.0 2212.0 0.9130271472854778 0
229705.0 33186.0 14.447225789599704 1
228691.0 27104.0 11.851800027110817 2
222104.0 31106.0 14.005150740193782 3
190137.0 13240.0 6.963400074682991 4
175981.0 27032.0 15.36074917178559 5
174098.0 7861.0 4.515273007156889 6
143114.0 3952.0 2.76143493997792 7
112725.0 6783.0 6.01729873586161 8
78055.0 2551.0 3.268208314649926 9


# VIS 3

In [10]:
change_data=[]
for country,group in df.groupby(['Country_Region']):
    temp_df=group.copy()
    temp_df['ConfirmedCasesChange']=temp_df['ConfirmedCases'].diff()
    temp_df['FatalitiesChange']=temp_df['Fatalities'].diff()
    lastmile=temp_df.ConfirmedCasesChange.rolling(window=7).mean().tolist()[-7:]
    temp={}
    temp['increasing']=False
    temp['Country']=country
    temp['ConfirmedCases']=temp_df['ConfirmedCases'].tolist()[-1]
    temp['Fatalities']=temp_df['Fatalities'].tolist()[-1]
    temp['ConfirmedCasesPercent']=lastmile[-1]-lastmile[-2]  
    if(temp['ConfirmedCasesPercent']> 0):
        temp['increasing']=True
    temp['ConfirmedCasesPercent']=abs(temp['ConfirmedCasesPercent'])
    change_data.append(temp)
    

In [11]:
change_data_df=pd.DataFrame(change_data)
# change_data_df=change_data_df.sort_values(['increasing','ConfirmedCases'],ascending=False)
fig = go.Figure()
count=0
change=len(change_data_df.loc[change_data_df.increasing == False])
for index,row in change_data_df.loc[change_data_df.increasing == False].iterrows():
    fig.add_trace(go.Barpolar(r=[row['ConfirmedCasesPercent']],theta=[90*count/change],name=row['Country'],marker=dict(color='red')))
    count+=1
count=0
change=len(change_data_df.loc[change_data_df.increasing == True])
for index,row in change_data_df.loc[change_data_df.increasing == True].iterrows():
    fig.add_trace(go.Barpolar(r=[row['ConfirmedCasesPercent']],theta=[90+90*count/change],name=row['Country'],marker=dict(color='blue')))    
    count+=1
fig.update_layout(polar = dict(
        domain = dict( # set chart size and position 
        x = [0, 1],
        y = [0, 1]),
        sector = [0,180],   # set chart shape (half or full)
        angularaxis = dict(
            thetaunit = "degrees",
            dtick = 10,
            #rotation = -90, #does not work
            #direction = "clockwise" # does not work
            )
        )
)
fig.show()    
change_data_df

,increasing,Country,ConfirmedCases,Fatalities,ConfirmedCasesPercent
0,True,Afghanistan,5226.0,132.0,13.571429
1,False,Albania,880.0,31.0,1.142857
2,True,Algeria,6253.0,522.0,3.857143
3,True,Andorra,760.0,49.0,0.285714
4,False,Angola,45.0,2.0,0.000000
...,...,...,...,...,...
179,False,Vietnam,288.0,0.0,0.000000
180,False,West Bank and Gaza,375.0,2.0,0.428571
181,False,Western Sahara,6.0,0.0,0.000000
182,False,Zambia,446.0,7.0,0.428571


In [12]:
change_data=[]
for country,group in df.groupby(['Country_Region']):
    temp_df=group.copy()
    temp_df['ConfirmedCasesChange']=temp_df['ConfirmedCases'].diff()
    temp_df['FatalitiesChange']=temp_df['Fatalities'].diff()
    temp_df['WeeklyChangeConfirmed']=temp_df.ConfirmedCasesChange.rolling(window=7).mean()
    temp_df['WeeklyChangeFatalities']=temp_df.FatalitiesChange.rolling(window=7).mean()
    lastmile=temp_df.WeeklyChangeConfirmed.tolist()[-7:]
    temp={}
    temp['increasing']=False
    temp['Country']=country
    temp['ConfirmedCases']=temp_df['ConfirmedCases'].tolist()[-1]
    temp['Fatalities']=temp_df['Fatalities'].tolist()[-1]
    WeeklyChangeConfirmed=temp_df['WeeklyChangeConfirmed'].tolist()
    WeeklyChangeFatalities=temp_df['WeeklyChangeFatalities'].tolist()
    temp['WeeklyChangeConfirmed']=100*(WeeklyChangeConfirmed[-1]-WeeklyChangeConfirmed[-2])/(WeeklyChangeConfirmed[-2]) if WeeklyChangeConfirmed[-2] != 0 else 0
    temp['WeeklyChangeFatalities']=100*(WeeklyChangeFatalities[-1]-WeeklyChangeFatalities[-2])/(WeeklyChangeFatalities[-2]) if WeeklyChangeFatalities[-2] != 0 else 0
    if(temp['WeeklyChangeConfirmed'] > 0):
        temp['increasing']=True
    temp['WeeklyChangeConfirmed']=abs(temp['WeeklyChangeConfirmed'])
    temp['WeeklyChangeFatalities']=abs(temp['WeeklyChangeFatalities'])
    change_data.append(temp)
change_data_df=pd.DataFrame(change_data)    
change_data_df=change_data_df.sort_values('WeeklyChangeConfirmed')


In [15]:

# change_data_df=change_data_df.sort_values(['increasing','ConfirmedCases'],ascending=False)
fig = go.Figure()
count=0
# change=len(change_data_df.loc[change_data_df.increasing == True])
change=10
for index,row in change_data_df.loc[change_data_df.increasing == True].tail(change).iterrows():
    fig.add_trace(go.Barpolar(r=[row['WeeklyChangeConfirmed']],theta=[90*count/change],name=row['Country'],marker=dict(color='red')))    
    count+=1

count=0
change=15
for index,row in change_data_df.loc[change_data_df.increasing == False].tail(change).iterrows():
    fig.add_trace(go.Barpolar(r=[row['WeeklyChangeConfirmed']],theta=[180-90*count/change],name=row['Country'],marker=dict(color='blue')))
    count+=1

fig.update_layout(polar = dict(
        domain = dict( # set chart size and position 
        x = [0, 1],
        y = [0, 1]),
        sector = [0,180],   # set chart shape (half or full)
#         angularaxis = dict(
#             thetaunit = "degrees",
#             dtick = 10,
#             ),
        radialaxis=dict(range=[0,100])
        )
)
fig.show()    
change_data_df

,increasing,Country,ConfirmedCases,Fatalities,WeeklyChangeConfirmed,WeeklyChangeFatalities
91,False,Kosovo,919.0,29.0,0.0,0.0
159,False,Suriname,10.0,1.0,0.0,0.0
32,False,Canada,11.0,0.0,0.0,0.0
157,False,Sri Lanka,915.0,9.0,0.0,0.0
36,False,China,1268.0,1.0,0.0,0.0
...,...,...,...,...,...,...
180,False,West Bank and Gaza,375.0,2.0,75.0,0.0
19,True,Bhutan,15.0,0.0,100.0,0.0
74,False,Holy See,12.0,0.0,100.0,0.0
98,False,Libya,64.0,3.0,100.0,0.0
